# 단어 변환

두 단어 begin, target 단어 집함 words
begin => target 변환하는 가장 짧은 변환 과정 찾기.

1. 한 번에 한 개의 알파벳만 바꿀 수 있다.
2. words에 있는 단어로만 변환할 수 있다.

예를 들어 begin이 hit, target가 cog, words가 [hot,dot,dog,lot,log,cog]라면 hit -> hot -> dot -> dog -> cog와 같이 4단계를 거쳐 변환할 수 있습니다.

두 개의 단어 begin, target과 단어의 집합 words가 매개변수로 주어질 때, 최소 몇 단계의 과정을 거쳐 begin을 target으로 변환할 수 있는지 return 하도록 solution 함수 작성

- 각 단어는 소문자
- 각 단어 길이 3이상 10이하, 모든 단어 길이 같음
- words에는 3개 이상 50개 이하 단어
- begin, target은 같지 않다.
- 변환 불가하면 0 return

# 시도 1: 2,3,4 번 시간초과
1. target이 words 안에 없다면 return 0
2. words 로 주어진 목록을 토대로 graph를 만든다.
3. 첫 level에 begin 문자열을 넣고 시작
4. 각 level 에서 current 값을 하나씩 꺼낸다.
5. current에서 한 글자 변경해서 만들 수 있는 문자열들을 다음 레벨에 포함시키고 words 목록에서 제거.
6. 그리고 만들어진 다음레벨에서 순서대로 4번부터 다시 진행하는 구조.
7. words 에 있는 문자열들을 다 사용하면 끝낸다.
8. 그리고 만들어진 graph에서 target을 대상으로 dfs 진행하고, visited 에 포함된 갯수 -1 이 답 ( begin 제외 )

In [26]:
from collections import defaultdict
def solution(begin, target, words):
    answer = 0
    
    if target not in words:
        return answer
    
    graph = defaultdict(str)

    def one_char_diff(s1, s2):
        count = 0

        for c1, c2 in zip(s1, s2):
            if c1 != c2:
                if count == 1:
                    return False
                else:
                    count+=1

        return True
     
    level = [begin]

    while len(words):    
        next_level = set()

        for current in level:
            # 중복되는 단어가 없기 때문에 set을 사용해도 된다.
            graph[current] = set()        

            for w in words:
                if one_char_diff(current,w):
                    graph[current].add(w)
                    next_level.add(w)            
            
            for sub in graph[current]:
                words.remove(sub)

        level = next_level

    # print(graph)
    
    def DFS_count_with_adj_list(graph, root):
        count = 0
        visited = []
        stack = [root]

        while stack:
            n = stack.pop()
            if n not in visited:
                visited.append(n)                
                # print(n,graph[n],target)
                if n == target:
                    break
                stack += graph[n] - set(visited)    
                count+=1                                                            

        return count

    answer = DFS_count_with_adj_list(graph,begin)
    
    return answer

In [27]:
solution('hit','cog',['hot', 'dot', 'dog', 'lot', 'log', 'cog'])

4